# Classes

There are two material classes to use `mat_lib` and `mat_map`, they are both contained in `material.py`.

In [1]:
from material import mat_lib, mat_map
from mesh import Cell, Mesh
import numpy as np

1. [Cell Readme](#Cell-Readme)
1. [Cell API](#Cell-API)
1. [Material Readme](#Material-Readme)
1. [Material API Summary](#Material-API)
1. [Mesh Readme](#Mesh-Readme)
1. [Mesh API Summary](#Mesh-API)


# Cell Readme

Initialize a cell using a tuple index, and a dictionary that contains mesh parameters required to create the cell.

In [2]:
index = (1,1)
mesh_params = {'x_cell': 4,  #REQ: number of cells along the x-axis
               'y_cell': 4,  #OP: cells on y-axis (def to x_cell)
               'cell_length': 2.5 #REQ: cell length
              }
cell = Cell(index, mesh_params)

Once initialized there are automatically calculated quantities accessed using function calls:

In [3]:
cell.index() # Cell index

(1, 1)

In [4]:
cell.global_idx() # Nodes

[6, 7, 11, 12]

In [5]:
cell.length()

2.5

In [6]:
cell.area()

6.25

If the cell is on a boundary, it will have a dictionary with the appropriate boundaries:

In [7]:
index2 = (4,4)
cell2 = Cell(index2, mesh_params)

In [8]:
cell2.bounds()

{'x_max': None, 'y_max': None}

If it is _not_ on a boundary, the dictionary will be empty, allowing you to use this to determine if it is a boundary cell:

In [9]:
for c in [cell, cell2]:     # cell is NOT a boundary, cell2 IS
    if c.bounds():
        print('cell at ' + str(c.index()) + ' is a boundary cell')
    if not c.bounds():
        print('cell at ' + str(c.index()) + ' is NOT a boundary cell')

cell at (1, 1) is NOT a boundary cell
cell at (4, 4) is a boundary cell


The bounds are `x_min`, `x_max`, `y_min`, and `y_max`. They are also set using the same `cell.bounds()` method:

In [10]:
cell2.bounds('x_max', 'refl')
print cell2.bounds()

{'y_max': None, 'x_max': 'refl'}


Trying to set a bound that it doesn't have will result in a key error:

In [11]:
cell2.bounds('x_min', 'refl')

KeyError: 'Cell does not have bound x_min'

If a cell is defined by passing a material map `mat_map`, you can access the material properties using the same syntax as the `mat_map.get` function:

In [12]:
file_loc = './tests/testData/materials/'
files = [file_loc + m for m in ['test_mat.xml', 'test_mat2.xml']]
lib = mat_lib(n_grps = 2, files = files)
layout = """1 1 1 1
            1 2 2 1
            1 2 2 1
            1 1 1 1"""
layout_dict = {'1':'test_mat2', '2': 'test_mat'}
mmap = mat_map(lib = lib, layout = layout, layout_dict = layout_dict, 
               x_max = 10, n=20)
index = (1,1)
mesh_params = {'x_cell': 4,  #REQ: number of cells along the x-axis
               'y_cell': 4,  #OP: cells on y-axis (def to x_cell)
               'cell_length': 2.5 #REQ: cell length
              }
cell = Cell(index, mesh_params, mat_map = mmap)

**MAKE SURE THE DIMENSIONS OF THE MATERIAL MAP MATCH THE MESH PARAMETERS OR THEY WON'T LINE UP**

In [13]:
print cell.get('sig_t')
print cell.get('id')

[ 20.  30.]
test_mat


# Cell API

Initialization:

In [14]:
index = (1,1)
mesh_params = {'x_cell': 4,  #REQ: number of cells along the x-axis
               'y_cell': 4,  #OP: cells on y-axis (def to x_cell)
               'cell_length': 2.5 #REQ: cell length
              }
cell = Cell(index, mesh_params)

Calculated Properties:

In [15]:
print cell.global_idx()
print cell.length()
print cell.area()

[6, 7, 11, 12]
2.5
6.25


If on a boundary, `cell.bounds()` will return a dictionary with the boundaries or an empty dictionary.

In [16]:
print cell.bounds()

{}


In [17]:
index2 = (4,4)
boundary_cell = Cell(index2, mesh_params)
boundary_cell.bounds()

{'x_max': None, 'y_max': None}

Cell bounds are also set or called using the bounds function:

In [18]:
boundary_cell.bounds('y_max', 'refl')
boundary_cell.bounds('x_max', 'vac')
print boundary_cell.bounds()
print boundary_cell.bounds('x_max')

{'y_max': 'refl', 'x_max': 'vac'}
vac


If you define a material map, material properties can be retrieved using `get`

In [19]:
file_loc = './tests/testData/materials/'
files = [file_loc + m for m in ['test_mat.xml', 'test_mat2.xml']]
lib = mat_lib(n_grps = 2, files = files)
layout = """1 1 1 1
            1 2 2 1
            1 2 2 1
            1 1 1 1"""
layout_dict = {'1':'test_mat2', '2': 'test_mat'}
mmap = mat_map(lib = lib, layout = layout, layout_dict = layout_dict, 
               x_max = 10, n=20)
index = (1,1)
mesh_params = {'x_cell': 4,  #REQ: number of cells along the x-axis
               'y_cell': 4,  #OP: cells on y-axis (def to x_cell)
               'cell_length': 2.5 #REQ: cell length
              }
cell = Cell(index, mesh_params, mat_map = mmap)

In [20]:
cell.get('sig_t')

array([ 20.,  30.])

# Material Readme

## Mat_lib

`mat_lib` is the **Material Library**, and holds all the problem materials. This class should be used when info about a specific material, or all the materials is required. 

To initialize, it requires a list of all the desired material `xml` files, and the number of energy groups in the desired group structure, `n_grps`:

In [21]:
file_loc = './tests/testData/materials/'
files = [file_loc + m for m in ['test_mat.xml', 'test_mat2.xml']]
lib = mat_lib(n_grps = 2, files = files)

The library can also be intialized with only the number of groups, and then materials added using `mat_lib.add`, this is probably more useful if iterating through all the files in a directory. `n_grps` is always required at initialization.

In [22]:
lib2 = mat_lib(n_grps = 2)
for f in files:
    lib2.add(f)

Two utility functions return info about the materials, `ids` returns the material IDs and `props` shows all the properties that each material has, or the properties of a specific material id.

In [23]:
lib.ids()

['test_mat', 'test_mat2']

In [24]:
lib.props()

{'test_mat': [['name', 'id'],
  ['nu'],
  ['chi', 'g_thermal'],
  ['sig_s', 'sig_a', 'sig_f', 'sig_t'],
  ['diff_coef_ua',
   'diff_coef',
   'chi_nu_sig_f',
   'inv_sig_t',
   'sig_t_ua',
   'ksi_ua']],
 'test_mat2': [['name', 'id'],
  ['nu'],
  ['chi', 'g_thermal'],
  ['sig_s', 'sig_a', 'sig_f', 'sig_t'],
  ['diff_coef_ua',
   'diff_coef',
   'chi_nu_sig_f',
   'inv_sig_t',
   'sig_t_ua',
   'ksi_ua']]}

In [25]:
lib.props('test_mat')

[['name', 'id'],
 ['nu'],
 ['chi', 'g_thermal'],
 ['sig_s', 'sig_a', 'sig_f', 'sig_t'],
 ['diff_coef_ua',
  'diff_coef',
  'chi_nu_sig_f',
  'inv_sig_t',
  'sig_t_ua',
  'ksi_ua']]

Number of groups can be accessed using two methods:

In [26]:
lib.n_grps()

2

In [27]:
lib.get('n_grps')

2

#### Accessing Data

The main method to access data is `get`. This will either return:
- A dictionary containing all the materials as keys, and the desired property as values **or**
- The value of the property for a particular material.

In [28]:
# Get dictionary of all sig_t values:
lib.get('sig_t')

{'test_mat': array([ 20.,  30.]), 'test_mat2': array([ 10.,  20.])}

In [29]:
# Get values for a specific material:
lib.get('sig_t', mat_id = 'test_mat')

array([ 20.,  30.])

Note this is identical to using:

In [30]:
lib.get('sig_t')['test_mat']

array([ 20.,  30.])

To get a property per steradian, use `get_per_str`:

In [31]:
lib.get_per_str('sig_t', mat_id='test_mat')

array([ 1.59154943,  2.38732415])

Other functions will return cell corrections and bd corrections using the same syntax:

In [32]:
print lib.cell_correction_ua(np.array([[1.0, 1.0],[1.0,1.0]]))
print lib.bd_correction_ua(np.array([1.0, 1.0]))

{'test_mat2': array([ 120.,  120.]), 'test_mat': array([ 70.,  70.])}
{'test_mat2': 120.0, 'test_mat': 70.0}


## MAT_MAP

`mat_map` is a **material mapping** that takes the problem material layout and connects it to the materials contained in the material library.

To make a mapping, it requires
- `lib` A material library that contains all problem materials
- `layout` A string showing the material layout of the problem, with different values meaning different materials. This must have a square number of values to map to the square problem.
- `layout_dict` a dictionary connecting the strings used in the layout to the material ids in the library.
- `x_max` maximum x-value of the problem.
- `n` number of cells along each side of the problem
OPTIONAL
- `x_min`, `y_min` otherwise defaults to 0
- `y_max` otherwise defaults to `x_max`

The layout string uses any characters to represent different materials, you can use any characters or strings that you want. For example, to have one material surrounded by another.

**Note:** The lower left of this array is the problem origin (0,0)

In [33]:
# Both of these are identical, 
# as any line breaks and extra spaces are removed. 
layout = """1 1 1 1
            1 2 2 1
            1 2 2 1
            1 1 1 1"""
layout2 = " 1 1 1 1 1 2 2 1 1 2 2 1 1 1 1 1"

Next, the arbirary characters used (1 and 2 here) need to be linked back to materials in your material library. For example, we have two materials in ours:

In [34]:
lib.ids()

['test_mat', 'test_mat2']

We link them using a dictionary, pairing the layout character with the material id:

In [35]:
layout_dict = {'1':'test_mat2', '2': 'test_mat'}

We can then make our mapping:

In [36]:
mmap = mat_map(lib = lib, layout = layout, layout_dict = layout_dict, 
               x_max = 4, n=4)

Now we can get material properties at any (x,y) value, or any cell index. Here, we get the name of the material, and the $\Sigma_t$ using x and y coordinates in a tuple:

In [37]:
print mmap.get('id', (0.5, 0.5))
print mmap.get('sig_t', (0.5, 0.5))

test_mat2
[ 10.  20.]


As expected, that location returns the correct material and properties. Cell index can also be used:

In [38]:
print mmap.get('id', 1)
print mmap.get('sig_t', 1)

test_mat2
[ 10.  20.]


Use the same names to get cell corrections:

In [39]:
mmap.cell_correction_ua(np.array([[1.0, 1.0],[1.0,1.0]]), (0.5, 0.5))

array([ 120.,  120.])

# Material API

This is a summary of all functions.

## mat_lib

In [3]:
file_loc = './tests/testData/materials/'
files = [file_loc + m for m in ['test_mat.xml', 'test_mat2.xml']]

In [4]:
# Initialize using list of files, set tr_scatt = True to transpose the
# scattering matrix.
lib = mat_lib(n_grps = 2, files = files) 

# Add new ones using add
lib2 = mat_lib(n_grps = 2)
for f in files:
    lib2.add(f)

In [42]:
# Get IDs of materials
lib.ids()

['test_mat', 'test_mat2']

In [43]:
# Get Properties of materials
lib.props()

{'test_mat': [['name', 'id'],
  ['nu'],
  ['chi', 'g_thermal'],
  ['sig_s', 'sig_a', 'sig_f', 'sig_t'],
  ['diff_coef_ua',
   'diff_coef',
   'chi_nu_sig_f',
   'inv_sig_t',
   'sig_t_ua',
   'ksi_ua']],
 'test_mat2': [['name', 'id'],
  ['nu'],
  ['chi', 'g_thermal'],
  ['sig_s', 'sig_a', 'sig_f', 'sig_t'],
  ['diff_coef_ua',
   'diff_coef',
   'chi_nu_sig_f',
   'inv_sig_t',
   'sig_t_ua',
   'ksi_ua']]}

In [44]:
# Get number of groups
lib.n_grps()

2

In [45]:
lib.get('n_grps')

2

In [46]:
# Get properties of a specific material
lib.props('test_mat')

[['name', 'id'],
 ['nu'],
 ['chi', 'g_thermal'],
 ['sig_s', 'sig_a', 'sig_f', 'sig_t'],
 ['diff_coef_ua',
  'diff_coef',
  'chi_nu_sig_f',
  'inv_sig_t',
  'sig_t_ua',
  'ksi_ua']]

In [47]:
#Get properties of all materials
lib.get('sig_t')

{'test_mat': array([ 20.,  30.]), 'test_mat2': array([ 10.,  20.])}

In [48]:
#Get property of a specific material
lib.get('sig_t', mat_id = 'test_mat')

array([ 20.,  30.])

In [49]:
# Calculate corrections:
print lib.cell_correction_ua(np.array([[1.0, 1.0],[1.0,1.0]]))
print lib.bd_correction_ua(np.array([1.0, 1.0]))

{'test_mat2': array([ 120.,  120.]), 'test_mat': array([ 70.,  70.])}
{'test_mat2': 120.0, 'test_mat': 70.0}


In [50]:
# can also get for a specific material
lib.cell_correction_ua(np.array([[1.0, 1.0],[1.0,1.0]]), 
                             mat_id='test_mat')

array([ 70.,  70.])

## mat_map

In [5]:
# Create layout and layout dictionary
layout = """1 1 1 1
            1 2 2 1
            1 2 2 1
            1 1 1 1"""
layout_dict = {'1':'test_mat2', '2': 'test_mat'}

# Initialize
mmap = mat_map(lib = lib, layout = layout, layout_dict = layout_dict, 
               x_max = 4, n=4)

In [52]:
# Get values at locations, (x,y) or cell index k
print mmap.get('id', (0.5, 0.5))
print mmap.get('sig_t', (0.5, 0.5))
print mmap.get('id', 1)
print mmap.get('sig_t', 1)

test_mat2
[ 10.  20.]
test_mat2
[ 10.  20.]


In [53]:
# Get corrections at locations using the same syntax
mmap.cell_correction_ua(np.array([[1.0, 1.0],[1.0,1.0]]), (0.5, 0.5))
mmap.cell_correction_ua(np.array([[1.0, 1.0],[1.0,1.0]]), 1)

array([ 120.,  120.])

# Mesh Readme

The `Mesh` class creates cells for a problem with a given number of cells per side and a total dimension.

A mesh requires:
- `mesh_cells` how many mesh cells per side of the domain
- `domain_upper` the upper bound of the domain (lower assumed to be 0)
- `mat_map` material mapping

Make sure your `mat_map` dimensions match the `domain_upper`. Here we use 10 for both `x_max` and `domain upper`

In [6]:
mmap = mat_map(lib = lib, layout = layout, layout_dict = layout_dict, 
               x_max = 10, n=4)
mesh_cells = 4
domain_upper = 10
mesh = Mesh(mesh_cells, domain_upper, mmap)

Now we can access many properties of the mesh:

In [7]:
print mesh.n_cell()
print mesh.n_node()

16
25


In [8]:
print mesh.x_cell()
print mesh.x_node()

4
5


In [9]:
print mesh.y_cell()
print mesh.y_node()

4
5


In [10]:
mesh.cell_length()

2.5

and can get all the cells:

In [11]:
mesh.cells()

In [13]:
for cell in mesh.cells():
    if cell.bounds():
        print 'Boundary cell ', cell.index(), cell.get('id'), cell.bounds()
    else:
        print 'Interior cell ', cell.index(), cell.get('id')

Boundary cell  (0, 0) test_mat2 {'x_min': None, 'y_min': None}
Boundary cell  (0, 1) test_mat2 {'x_min': None}
Boundary cell  (0, 2) test_mat2 {'x_min': None}
Boundary cell  (0, 3) test_mat2 {'x_min': None, 'y_max': None}
Boundary cell  (1, 0) test_mat2 {'y_min': None}
Interior cell  (1, 1) test_mat
Interior cell  (1, 2) test_mat
Boundary cell  (1, 3) test_mat2 {'y_max': None}
Boundary cell  (2, 0) test_mat2 {'y_min': None}
Interior cell  (2, 1) test_mat
Interior cell  (2, 2) test_mat
Boundary cell  (2, 3) test_mat2 {'y_max': None}
Boundary cell  (3, 0) test_mat2 {'y_min': None, 'x_max': None}
Boundary cell  (3, 1) test_mat2 {'x_max': None}
Boundary cell  (3, 2) test_mat2 {'x_max': None}
Boundary cell  (3, 3) test_mat2 {'y_max': None, 'x_max': None}
